In [1]:
    from wtypes import *
    import pytest, anyconfig
    invalid = pytest.raises(ValidationError)

    from wtypes import *
    import pytest, anyconfig
    invalid = pytest.raises(ValidationError)

In [2]:
    def test_dict():
        class Thing(Dict):
            a: Integer

        assert Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        assert Thing(a=1)
        with invalid:
            Thing(a='abc')


    def test_dict():
        class Thing(Dict):
            a: Integer

        assert Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        assert Thing(a=1)
        with invalid:
            Thing(a='abc')

In [3]:
    def test_bunch():
        class Thing(Bunch):
            a: Integer

        assert Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        t = Thing(a=1)
        assert t.a == 1
        with invalid:
            Thing(a='abc')


    def test_bunch():
        class Thing(Bunch):
            a: Integer

        assert Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        t = Thing(a=1)
        assert t.a == 1
        with invalid:
            Thing(a='abc')

In [4]:
    def test_dict_mods():
        d = Dict[{'a': Integer, 'b': String}]({'a': 1})

        with invalid: d["a"] = 'abc'
        d['a'] = 8
        d['b'] = 'abc'
        with invalid: d['b'] = 10
        d['b'] = 'wxyz'

        assert d == {'a': 8, 'b': 'wxyz'}


    def test_dict_mods():
        d = Dict[{'a': Integer, 'b': String}]({'a': 1})

        with invalid: d["a"] = 'abc'
        d['a'] = 8
        d['b'] = 'abc'
        with invalid: d['b'] = 10
        d['b'] = 'wxyz'

        assert d == {'a': 8, 'b': 'wxyz'}

In [5]:
    def test_list_mods():
        l = List[Integer, Bool]()
        with invalid: l.append(1.2)
        l.append(1)
        l.extend([1,True])
        with invalid: l.extend([1, 'abc'])
        l.extend([False, 2, 4])
        assert l == [1, 1, True, False, 2, 4]
        with invalid: l.insert(1, 'abc')
        l.insert(1, 8)
        assert l == [1, 8, 1, True, False, 2, 4]

        ml = List[Integer] + MinItems[1]

        with invalid: ml()
        with invalid: ml(['abc'])
        assert ml([1])

    def test_list_mods():
        l = List[Integer, Bool]()
        with invalid: l.append(1.2)
        l.append(1)
        l.extend([1,True])
        with invalid: l.extend([1, 'abc'])
        l.extend([False, 2, 4])
        assert l == [1, 1, True, False, 2, 4]
        with invalid: l.insert(1, 'abc')
        l.insert(1, 8)
        assert l == [1, 8, 1, True, False, 2, 4]

        ml = List[Integer] + MinItems[1]

        with invalid: ml()
        with invalid: ml(['abc'])
        assert ml([1])

In [6]:
    def test_update_item():
        for cls in [Dict, Bunch]:
            d = cls[{'a': int}]()
            assert 'a' not in d
            d['a'] = 10
            assert d['a'] == 10

    def test_update_item():
        for cls in [Dict, Bunch]:
            d = cls[{'a': int}]()
            assert 'a' not in d
            d['a'] = 10
            assert d['a'] == 10

In [7]:
    def test_load_config():
        class BadProject(Bunch):
            tool: Dict
            __annotations__ = {'build-system': Integer}

        with invalid:
            BadProject().load('pyproject.toml')
        
        class PyProject(Bunch):
            tool: Dict
            __annotations__ = {'build-system': Dict}
        PyProject.from_config_file('pyproject.toml')

    def test_load_config():
        class BadProject(Bunch):
            tool: Dict
            __annotations__ = {'build-system': Integer}

        with invalid:
            BadProject().load('pyproject.toml')
        
        class PyProject(Bunch):
            tool: Dict
            __annotations__ = {'build-system': Dict}
        PyProject.from_config_file('pyproject.toml')

In [8]:
    def test_tuple():
        assert Tuple((1,2))
        assert Tuple([1,2])
        with invalid: Tuple[[Integer, String]]({})

    def test_tuple():
        assert Tuple((1,2))
        assert Tuple([1,2])
        with invalid: Tuple[[Integer, String]]({})

In [9]:
    def test_dataclass():
        class Thing(DataClass):
            a: Integer

        assert dataclasses.is_dataclass(Thing)
        assert Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        assert Thing(a=1)
        with invalid:
            Thing(a='abc')


    def test_dataclass():
        class Thing(DataClass):
            a: Integer

        assert dataclasses.is_dataclass(Thing)
        assert Thing._schema.toDict() == {'type': 'object', 'properties': {'a': {'type': 'integer'}}, 'required': ['a']}
        assert Thing(a=1)
        with invalid:
            Thing(a='abc')

In [10]:
    def test_update_attr():
        
        for cls in [Bunch]:
            class Class(cls): 
                a: int = None
            d = Class()
            with invalid: d['a'] = 'abc'
            assert 'a' not in d
            d.a = 10
            assert d.a == 10

    def test_update_attr():
        
        for cls in [Bunch]:
            class Class(cls): 
                a: int = None
            d = Class()
            with invalid: d['a'] = 'abc'
            assert 'a' not in d
            d.a = 10
            assert d.a == 10